# P1 Examples — Single-Ion Dynamics (Mg⁺ & Ba⁺)
This notebook reproduces the Mg⁺ and Ba⁺ reference calculations for Pillar 1. See the [theory notes](../P1_single_ion_dynamics.md) for derivations and context.

In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt

# Physical constants (SI)
e = 1.602176634e-19
hbar = 1.054571817e-34
kB = 1.380649e-23

# Species masses (kg)
m_Mg = 3.984e-26     # 24Mg+
m_Ba = 2.293e-25     # 138Ba+


In [ ]:
def q_param(Vrf, r0, Omega_rf, m):
    return 2 * e * Vrf / (m * r0**2 * Omega_rf**2)

def omega_sec(Omega_rf, q, a=0.0):
    """Secular frequency in the small-q limit.
    Parameters
    ----------
    Omega_rf : float
        RF drive angular frequency (rad/s).
    q : array_like
        Mathieu q parameter.
    a : float, optional
        Mathieu a parameter (defaults to 0 for radial modes).
    """
    return 0.5 * Omega_rf * np.sqrt(a + 0.5 * np.square(q))

def emm_amplitude_direct(Edc, m, Omega_rf):
    """Excess micromotion amplitude for a stray DC field Edc."""
    return e * Edc / (m * Omega_rf**2)

def emm_energy_T(u_mm, m, Omega_rf):
    """Cycle-averaged micromotion energy converted to kelvin."""
    Emm = 0.25 * m * Omega_rf**2 * np.square(u_mm)
    return Emm / kB

def x0(hbar, m, omega):
    return math.sqrt(hbar / (2 * m * omega))

def eta(lambda_m, x0_m):
    k = 2 * math.pi / lambda_m
    return k * x0_m


In [ ]:
# Example trap
Vrf = 300.0       # V
r0 = 0.5e-3       # m
Omega_rf = 2 * math.pi * 20e6  # rad/s
Edc = 100.0       # V/m

for name, m in [("Mg+", m_Mg), ("Ba+", m_Ba)]:
    q = q_param(Vrf, r0, Omega_rf, m)
    w = omega_sec(Omega_rf, q, a=0.0)
    u = emm_amplitude_direct(Edc, m, Omega_rf)
    Tmm_mK = 1e3 * emm_energy_T(u, m, Omega_rf)

    print(f"{name}: q={q:.3f}, omega_sec/2π={w / (2 * math.pi):.3f} MHz, u_mm={u * 1e9:.2f} nm, <E_mm>= {Tmm_mK:.2f} mK")

# Zero-point motion and Lamb–Dicke parameters at 1 MHz
omega_1MHz = 2 * math.pi * 1e6
x0_Mg = x0(hbar, m_Mg, omega_1MHz)
x0_Ba = x0(hbar, m_Ba, omega_1MHz)
eta_Mg_280 = eta(280e-9, x0_Mg)
eta_Ba_493 = eta(493e-9, x0_Ba)
eta_Ba_1762 = eta(1762e-9, x0_Ba)

print(f"\nZero-point @1 MHz: x0(Mg+)={x0_Mg * 1e9:.2f} nm, x0(Ba+)={x0_Ba * 1e9:.2f} nm")
print(f"Lamb–Dicke: eta(Mg+,280nm)={eta_Mg_280:.3f}, eta(Ba+,493nm)={eta_Ba_493:.3f}, eta(Ba+,1762nm)={eta_Ba_1762:.4f}")


In [ ]:
%matplotlib inline
Vrf_grid = np.linspace(50, 500, 200)

def sec_freq_curve(m):
    qg = q_param(Vrf_grid, r0, Omega_rf, m)
    return omega_sec(Omega_rf, qg) / (2 * np.pi)

plt.figure()
plt.plot(Vrf_grid, q_param(Vrf_grid, r0, Omega_rf, m_Mg), label="q (Mg+)")
plt.plot(Vrf_grid, q_param(Vrf_grid, r0, Omega_rf, m_Ba), label="q (Ba+)")
plt.xlabel("Vrf (V)")
plt.ylabel("q")
plt.legend()
plt.title("Stability parameter q vs Vrf")
plt.show()

plt.figure()
plt.plot(Vrf_grid, sec_freq_curve(m_Mg) / 1e6, label="ω⊥/2π (Mg+)")
plt.plot(Vrf_grid, sec_freq_curve(m_Ba) / 1e6, label="ω⊥/2π (Ba+)")
plt.xlabel("Vrf (V)")
plt.ylabel("Frequency (MHz)")
plt.legend()
plt.title("Secular frequency vs Vrf")
plt.show()

Edc_grid = np.linspace(1, 200, 200)

def Tmm_curve(m):
    u = emm_amplitude_direct(Edc_grid, m, Omega_rf)
    return 1e3 * emm_energy_T(u, m, Omega_rf)

plt.figure()
plt.plot(Edc_grid, Tmm_curve(m_Mg), label="<E_mm> (Mg+)")
plt.plot(Edc_grid, Tmm_curve(m_Ba), label="<E_mm> (Ba+)")
plt.xlabel("Edc (V/m)")
plt.ylabel("<E_mm> (mK)")
plt.legend()
plt.title("Micromotion energy vs stray field")
plt.show()


## Notes
- Assumes $a ightarrow 0$ for radial confinement and the small-$q$ pseudopotential approximation.
- Change $V_{\text{rf}}$, $r_0$, $\Omega_{\text{rf}}$, or $E_{\text{dc}}$ above to explore different trap settings.
- Pillar 2 and 3 notebooks will extend these calculations to collisions and multi-ion chains.